# LSTM sin embeddings

Modelo básico con embeddings entrenados acá. Un baseline :-)

In [1]:
import keras

Using TensorFlow backend.


In [2]:
%ls ../data/dev_es/

dev_es.tsv  train_es.tsv


In [3]:
import pandas as pd
df_train = pd.read_table("../data/dev_es/train_es.tsv", index_col="id")
df_dev = pd.read_table("../data/dev_es/dev_es.tsv", index_col="id")

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 100000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(num_words, embedding_vector_length, input_length=max_length))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 32)            3200000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               12928     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,266,257
Trainable params: 3,266,257
Non-trainable params: 0
_________________________________________________________________


In [6]:
loss, accuracy = model.evaluate(X_dev, y_dev)

print("Loss function: {:.3f}".format(loss))
print("Accuracy: {:.3f}%".format(accuracy*100))

500/500 [==============================] - 0s 586us/step
Loss function: 2.065
Accuracy: 73.400%


## Bidirectional LSTM

In [7]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(num_words, embedding_vector_length, input_length=max_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 32)            3200000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               106400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 3,332,257
Trainable params: 3,332,257
Non-trainable params: 0
_________________________________________________________________


In [8]:
loss, accuracy = model.evaluate(X_dev, y_dev)

print("Loss function: {:.3f}".format(loss))
print("Accuracy: {:.3f}%".format(accuracy*100))

500/500 [==============================] - 1s 1ms/step
Loss function: 1.997
Accuracy: 73.600%
